<a href="https://colab.research.google.com/github/Pujitha2016/Fit-Pulse-Anomaly-Detection-from-Fitness-Devices/blob/main/Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv("/content/CLEAN__FITPULSE__DATA__.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.head()


,timestamp,heart_rate,resting_heart_rate,daily_steps,hours_sleep,stress_level,anomaly_label,id
0,2024-01-01 00:00:00,74,55,6751,7.06,2.9,0,1
1,2024-01-01 00:01:00,71,58,5587,6.79,4.0,0,1
2,2024-01-01 00:02:00,75,60,6077,7.06,2.5,0,1
3,2024-01-01 00:03:00,80,60,5630,7.27,2.4,0,1
4,2024-01-01 00:04:00,71,59,5652,7.02,2.7,0,1


In [2]:
!pip install prophet


In [3]:
from prophet import Prophet

# Prepare data for Prophet
prophet_df = df[['timestamp', 'heart_rate']]
prophet_df.columns = ['ds', 'y']

model = Prophet()
model.fit(prophet_df)

forecast = model.predict(prophet_df)

# Residuals
df['predicted_hr'] = forecast['yhat']
df['residual_hr'] = df['heart_rate'] - df['predicted_hr']

# Residual-based anomaly
threshold = 2 * df['residual_hr'].std()
df['residual_anomaly'] = abs(df['residual_hr']) > threshold


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [4]:
df['threshold_anomaly'] = (
    (df['heart_rate'] < 50) |
    (df['heart_rate'] > 120) |
    (df['hours_sleep'] < 4) |
    (df['hours_sleep'] > 10) |
    (df['stress_level'] > 8)
)


In [5]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

features = df[['heart_rate', 'hours_sleep', 'stress_level']]
scaled = StandardScaler().fit_transform(features)

kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster'] = kmeans.fit_predict(scaled)

# Distance from cluster center
import numpy as np
centers = kmeans.cluster_centers_
distances = np.linalg.norm(scaled - centers[df['cluster']], axis=1)

df['cluster_anomaly'] = distances > np.percentile(distances, 95)


In [6]:
df['final_anomaly'] = (
    df['residual_anomaly'] |
    df['threshold_anomaly'] |
    df['cluster_anomaly']
)



In [7]:
# Convert boolean to integer label
df['anomaly_label'] = df['final_anomaly'].astype(int)

In [8]:
# Check how many anomalies detected
df['anomaly_label'].value_counts()


,count
anomaly_label,
0,380
1,20


In [9]:
import plotly.express as px
import plotly.graph_objects as go

# Separate anomalies
hr_anomalies = df[df['anomaly_label'] == 1]

fig = px.line(
    df,
    x='timestamp',
    y='heart_rate',
    title='Heart Rate Time-Series with Anomalies'
)

# Add anomaly markers
fig.add_trace(
    go.Scatter(
        x=hr_anomalies['timestamp'],
        y=hr_anomalies['heart_rate'],
        mode='markers',
        marker=dict(color='red', size=7),
        name='Anomaly'
    )
)



fig.show()


In [10]:
sleep_anomalies = df[
    (df['hours_sleep'] < 4) | (df['hours_sleep'] > 10)
]

fig = px.line(
    df,
    x='timestamp',
    y='hours_sleep',
    title='Sleep Pattern Anomalies'
)

# Highlight sleep anomalies
fig.add_trace(
    go.Scatter(
        x=sleep_anomalies['timestamp'],
        y=sleep_anomalies['hours_sleep'],
        mode='markers',
        marker=dict(color='orange', size=7),
        name='Sleep Anomaly'
    )
)



fig.show()
